In [ ]:
#| default_exp utils

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import cv2
import torch
import numpy as np
from fastai.vision.core import TensorPoint, TensorBBox
from fastai.torch_core import tensor

In [ ]:
#| export
def check_device(inp, device):
    #may be another better way to do this?
    if not inp.device==device:
        inp=inp.to(device)
    return inp
    
def calc_area_torch(bbox):
    zeros= torch.zeros(1)
    zeros= check_device(zeros, bbox.device)
    return torch.maximum(bbox[:, :, 2]-bbox[:, :, 0], zeros)*\
           torch.maximum(bbox[:, :, 3]-bbox[:, :, 1], zeros)

def calc_pairwise_iou_torch(b1, b2):
    b1_inters=torch.max(b1[:, :, :2], b2[:, :, :2])
    b2_inters=torch.min(b1[:, :, 2:], b2[:, :, 2:])
    inters_coor = torch.cat((b1_inters, b2_inters), dim=-1)
    inters_area= calc_area_torch(inters_coor)
    area_1=calc_area_torch(b1)
    area_2=calc_area_torch(b2)
    union=area_1+area_2
    iou = inters_area / union.squeeze()
    return iou

def assign_anchor_torch(all_anchors, gt_bboxes, pos_iou_thr=0.7, neg_iou_thr=0.3):
    """
    all_anchors.shape= (1,512,4)
    gt_bboxes.shape= (1,1,4)
    """
    all_anchors= all_anchors.repeat(gt_bboxes.shape[0],1,1)
    pair_ious= calc_pairwise_iou_torch(all_anchors, gt_bboxes)
    flags= torch.full((all_anchors.shape[1],), fill_value=-1)
    flags= check_device(flags, all_anchors.device)
    for idx, _ in enumerate(gt_bboxes):
        valid_ind= torch.where(flags==-1)
        iou= pair_ious[idx, valid_ind[0]]
        pos_ind= torch.where((iou>=pos_iou_thr)|
                             (iou==iou.max()))
        pos_ind=torch.select(pos_ind[0], 0, 0)
        flags[pos_ind]=1

    max_iou_value, max_iou_inds= pair_ious.max(dim=0)
    neg= max_iou_value<neg_iou_thr
    neg_ind= torch.where((neg==True)&(flags==-1))
    flags[neg_ind]= 0
    return flags

def sample_anchor_torch(anchors, flags, num_per_img=128, pos_fraction=0.5):
    anchor_labels= torch.full((anchors.shape[0],),-1)
    anchor_labels= check_device(anchor_labels, anchors.device)
    fg_ind= torch.where(flags==1)[0]
    bg_ind= torch.where(flags==0)[0]
    fg_lim= int(num_per_img*pos_fraction)
    if fg_ind.shape[0] > fg_lim:
        ind= torch.randperm(fg_ind.shape[0])[:fg_lim]
        fg_ind= fg_ind[ind]
    # bg_lim= int(num_per_img-fg_ind.shape[0])
    bg_lim= int(num_per_img-fg_lim)
    ind= torch.randperm(bg_ind.shape[0])[:bg_lim]
    bg_ind= bg_ind[ind]
    anchor_labels[fg_ind]=1
    anchor_labels[bg_ind]=0
    return anchor_labels

def calc_offset_torch(gt_bboxes, anchors):
    cvt_bbox= xyxy2xywh(gt_bboxes)
    cvt_anchors= xyxy2xywh(anchors)
    x_del= (cvt_bbox[:, 0]-cvt_anchors[:, 0])/cvt_anchors[:, 2]
    y_del= (cvt_bbox[:, 1]-cvt_anchors[:, 1])/cvt_anchors[:, 3]
    w_del= torch.log((cvt_bbox[:, 2])/(cvt_anchors[:, 2]))
    h_del= torch.log((cvt_bbox[:, 3])/(cvt_anchors[:, 3]))
    return torch.stack([x_del, y_del, w_del, h_del]).T

def reverse_offset(anchors, offsets):
    cvt_anchors= xyxy2xywh(anchors)
    x= offsets[:, 0]*cvt_anchors[:, 2]+cvt_anchors[:, 0]
    y= offsets[:, 1]*cvt_anchors[:, 3]+cvt_anchors[:, 1]
    w= torch.exp(offsets[:, 2])*cvt_anchors[:, 2]
    h= torch.exp(offsets[:, 3])*cvt_anchors[:, 3]
    bbox=torch.stack([x,y,w,h]).T
    bbox=xywh2xyxy(bbox)
    return bbox

In [ ]:
#| export
def xyxy2xywh(bbox):
    w= bbox[:, 2]- bbox[:, 0]
    h= bbox[:, 3]- bbox[:, 1]
    x= bbox[:, 0]+ (w/2)
    y= bbox[:, 1]+ (h/2)
    if isinstance(bbox, torch.Tensor):
        bboxes= torch.stack([x,y,w,h]).T
    else: bboxes= np.vstack([x,y,w,h]).T
    return bboxes


def xywh2xyxy(bbox):
    x1= bbox[:, 0]- (bbox[:, 2]/2)
    y1= bbox[:, 1]- (bbox[:, 3]/2)
    x2= bbox[:, 0]+ (bbox[:, 2]/2)
    y2= bbox[:, 1]+ (bbox[:, 3]/2)
    if isinstance(bbox, torch.Tensor):
        bboxes= torch.stack([x1,y1,x2,y2]).T
    else: bboxes= np.vstack([x1,y1,x2,y2]).T
    return bboxes

def unscale_pnts(y, sz): 
    return TensorPoint((y+1) * tensor(sz).float()/2, img_size=sz)

def reg2loc(reg_feats):
    n= reg_feats.shape[0]
    return reg_feats.permute(0,2,3,1).contiguous().view(n,-1,4)

def cls_reshape(cls_feats, num_cats=2):
    n= cls_feats.shape[0]
    return cls_feats.permute(0,2,3,1).flatten(start_dim=1, end_dim=2).reshape(n,-1,2)

def nms(preds, iou_thr):
    x1 = preds[:, 0]
    y1 = preds[:, 1]
    x2 = preds[:, 2]
    y2 = preds[:, 3]
    scores = preds[:, 4]
    zeros= torch.zeros(1)
    zeros= check_device(zeros, preds.device)

    areas = torch.maximum((x2-x1), zeros) * \
            torch.maximum((y2-y1), zeros)
    # areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    scores_val, sorted_idx = scores.sort(descending=True)
    keep = []
    while sorted_idx.size(0) > 0:
        i = sorted_idx[0]
        keep.append(i)

        xx1 = torch.maximum(x1[i], x1[sorted_idx[1:]])
        yy1 = torch.maximum(y1[i], y1[sorted_idx[1:]])
        xx2 = torch.minimum(x2[i], x2[sorted_idx[1:]])
        yy2 = torch.minimum(y2[i], y2[sorted_idx[1:]])

        w = torch.maximum(xx2 - xx1 + 1, zeros)
        h = torch.maximum(yy2 - yy1 + 1, zeros)
        inter = w * h
        iou = inter / (areas[i] + areas[sorted_idx[1:]] - inter)

        retained_idx = torch.where(iou <= iou_thr)[0]
        sorted_idx = sorted_idx[retained_idx + 1]

    return preds[keep, :]

def draw_bboxes(img, bbox, identities=None, offset=(0,0), score_thr=0.5):
    for i,box in enumerate(bbox):
        x1,y1,x2,y2 = [int(i) for i in box]
        x1 += offset[0]
        x2 += offset[0]
        y1 += offset[1]
        y2 += offset[1]
        # box text and bar
        color = (0, 255, 10)
        cv2.rectangle(img,(x1, y1),(x2,y2),color,2)
        cv2.rectangle(img,(x1, y1),(x2,y2),(0,0,0),1)
    return img 


In [ ]:
#| export
def decode_rpn_preds(preds, all_anchors):
    rpn_reg_feats, rpn_cls_feats, rois= preds[:3]
    rois= rois[:, 1:].contiguous()
    bboxes=[]
    for idx,_ in enumerate(rpn_reg_feats):
        fg_score=rpn_cls_feats[idx,:,1]
        fg_max_score, fg_max_ind= fg_score.sort(descending=True)
        fg_max_ind= fg_max_ind[:1]
        bbox=reverse_offset(all_anchors[fg_max_ind], 
                            rpn_reg_feats[idx, fg_max_ind])
        bboxes.append(bbox)
    bboxes=torch.stack(bboxes)
    return TensorBBox(bboxes)

def decode_roi_preds(preds, all_anchors):
    rois, roi_reg_feats, roi_cls_feats= preds[2:]
    max_score, max_ind= roi_cls_feats.max(dim=-1)
    max_score= max_score.unsqueeze(-1)
    rois= rois[:,:,1:].contiguous()
    bboxes=[]
    for idx,_ in enumerate(roi_reg_feats):
        fg_max_score, fg_max_ind= max_score.sort(descending=True, dim=1)
        fg_max_ind= fg_max_ind[idx, :1]
        bbox= reverse_offset(rois[idx,fg_max_ind.squeeze(1)], roi_reg_feats[idx,fg_max_ind.squeeze(1)])
        # bbox= torch.clamp(bbox, min=0, max=256)
        bbox= torch.hstack([bbox, fg_max_score[idx, :1]])
        bboxes.append(TensorBBox(bbox))
    return torch.stack(bboxes)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()